<a href="https://colab.research.google.com/github/NgaAdrain/TEAM_Enter_Text_Here/blob/master/Data_Varification/20200707_Test_Model_4_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

csv -> data<br>



In [1]:
# Import the TensorFlow
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
tf.__version__


'2.2.0'

In [2]:
WINDOW_SIZE = 32
data_file = pd.read_csv('/content/20200707_161530.csv',encoding='utf-8')
data_file.drop(['Timestamp'],axis = 'columns',inplace=True)
#print(data_file)
#for d in data_file:
#  print(data_file)

In [3]:
label = data_file.pop('Label')
#list(data_file.values)

In [4]:
train_file = pd.read_csv('/content/20200707_160439.csv',encoding='utf-8')
train_file.RPM = train_file.RPM/5000
train_file.Velocity = train_file.Velocity/200
train_file.drop(['Timestamp'],axis = 'columns',inplace=True)
train_file_label = train_file.pop('Label1')

In [5]:
train_array = np.delete(train_file.values,-1,0)
train_label = np.delete(train_file_label.values,-1,0)

In [6]:
test_file = pd.read_csv('/content/20200707_164523.csv',encoding='utf-8')
test_file.drop(['Label1'],axis = 'columns',inplace=True)
test_file.RPM = test_file.RPM/5000
test_file.Velocity = test_file.Velocity/200
test_file_timestamp = test_file.pop('Timestamp')
test_array = np.delete(test_file.values,-1,0)

In [7]:
xdataset = []
ydataset = []
#in range()
for i in range(WINDOW_SIZE, train_array.shape[0] + 1, 1):
  xdataset.append(train_array[i-WINDOW_SIZE:i])
  ydataset.append(train_label[i-1])
xtrain = np.array(xdataset) #32개의 데이터를 1개의 window 단위로 설정
ytrain = np.array(ydataset) #각 window의 마지막 원소를 대표 label로 지정
print(xtrain.shape)
print(ytrain.shape)

(6149, 32, 7)
(6149,)


In [8]:
xdataset = []
ydataset = []
#in range()
for i in range(WINDOW_SIZE, test_array.shape[0] + 1, 1):
  xdataset.append(test_array[i-WINDOW_SIZE:i])
  ydataset.append(0)
xtest = np.array(xdataset) #32개의 데이터를 1개의 window 단위로 설정
ytest = np.array(ydataset)
print(xtest.shape)
print(ytest.shape)

(17867, 32, 7)
(17867,)


모든 value가 0\~1사이(또는 -1~1 사이)의 값이 되도록 나누기(정규화 과정)

In [9]:
data_file.RPM = data_file.RPM/5000
data_file.Velocity = data_file.Velocity/200

test_dataset의 sliding window를 정하기 ex) 32 -> 2차원을 3차원으로

In [10]:
print(type(data_file))
print(type(data_file.values))
np_array = np.delete(data_file.values,-1,0)
label = np.delete(label.values,-1,0)
print(type(np_array))

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [11]:
xdataset = []
ydataset = []
#in range()
for i in range(WINDOW_SIZE, np_array.shape[0] + 1, 1):
  xdataset.append(np_array[i-WINDOW_SIZE:i])
  ydataset.append(label[i-1])
np_xtest = np.array(xdataset) #32개의 데이터를 1개의 window 단위로 설정
np_ytest = np.array(ydataset) #각 window의 마지막 원소를 대표 label로 지정
print(np_xtest.shape)
print(np_ytest.shape)
#test_dataset = tf.data.Dataset.from_tensor_slices((data_file.values, label.values))
#print(type(data_file.values[0]))
#print(type(test_dataset))
#test_dataset = test_dataset.window(32,shift=1)
#data_file.values.shape
#for ds in test_dataset:
#  print(ds)

(2906, 32, 7)
(2906,)


In [12]:
#for val, lab in test_dataset.take(40):
#  print('Values: {}, Label: {}'.format(val,lab))
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(32, 7)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.fit(np_xtest, np_ytest, epochs=5)

Epoch 1/5
91/91 [==============================] - 0s 1ms/step - loss: 0.1838 - accuracy: 0.9446
Epoch 2/5
91/91 [==============================] - 0s 1ms/step - loss: 0.0397 - accuracy: 0.9900
Epoch 3/5
91/91 [==============================] - 0s 2ms/step - loss: 0.0233 - accuracy: 0.9945
Epoch 4/5
91/91 [==============================] - 0s 1ms/step - loss: 0.0159 - accuracy: 0.9962
Epoch 5/5
91/91 [==============================] - 0s 1ms/step - loss: 0.0103 - accuracy: 0.9972


In [14]:
model.fit(xtrain, ytrain, epochs=5)

Epoch 1/5
193/193 [==============================] - 0s 1ms/step - loss: 0.0182 - accuracy: 0.9950
Epoch 2/5
193/193 [==============================] - 0s 1ms/step - loss: 0.0059 - accuracy: 0.9987
Epoch 3/5
193/193 [==============================] - 0s 1ms/step - loss: 0.0031 - accuracy: 0.9993
Epoch 4/5
193/193 [==============================] - 0s 1ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 5/5
193/193 [==============================] - 0s 1ms/step - loss: 0.0010 - accuracy: 1.0000


In [33]:
#test_loss, test_acc = model.evaluate(xtest,  ytest, verbose=2)
predictions = model.predict(xtest)
count = 0
print(type(test_file_timestamp[0]))
txt = 'Timestamp,Direction\n'
for temp in xtest:
  #print(test_file_timestamp[count],end=' ')
  txt = txt + ',' + test_file_timestamp[count] + ' '
  if(np.argmax(predictions[count]) == 0):
  #  print('straight')
    txt = txt + ',straight\n'
  elif(np.argmax(predictions[count]) == 1):
  #  print('left')
    txt = txt + ',left\n'
  else:
    txt = txt + ',right\n'
  #  print('right')
  count = count + 1

f = open("test_output.csv",'w')
f.write(txt)
f.close()

<class 'str'>
